In [1]:
import numpy as np
import pandas as pd

from collections import Counter

from sklearn.model_selection import train_test_split

from keras.utils.np_utils import to_categorical

from keras import models
from keras import layers
from keras.datasets import imdb



In [2]:
(X_train,y_train), (X_test,y_test) =  imdb.load_data()

In [3]:
max_key = max(max([max(sentence) for sentence in X_train]),max([max(sentence) for sentence in X_test]))
max_key

88586

In [4]:
word_index = imdb.get_word_index()
max_vlaue = max(word_index.values())
max_vlaue

88584

In [5]:
index_word = {v:k for k,v in word_index.items()}

In [6]:
"""
make_sentences(X_train):
input : np.array: ([123,12,15,48],[51,74,63], ...)
output : list:['text', 'text', ...]


동작

np 어레이를 리스트로 typecasting 후
내부 리스트 하나씩(sentence) 읽어와서
원소(index) 하나씩 단어사전(index_word) 참조
만약 index가 사전에 최댓값 보다 크다면 
사전 참조하지 않고 continue
단어 단위를 더할때에는 사이에 공백(' ')추가
리스트에 문장을 넣을 때 양 끝에 공백 벗기고(.strip())추가
리스트 반환

"""
def make_sentences(dummy):
    dummy_list = []
    for sentence in list(dummy):
        context = ''
        for index in sentence:
            if index > max_vlaue:
                continue
            context += index_word[index]+' '
        dummy_list.append(context.strip())
    return dummy_list

"""

contexts_to_index(sentence_list):
input: text_list: ['abc asv', 'bdvs brrwab brb', ...] 
output: index_list: [[1594, 2456, 24154, 1514, 1248, 12],[154,87,7,213,8], ...]

동작
인덱스리스트의 내부 리스트(sentence)를 하나씩 가져와서
만약 sentence에 공백이 존재하면 (하나 이상의 단어라는 의미)
word_list를 sentence.split으로 생성 단어 리스트
단어리스트를 보면서 index_word를 사전참조해
tmp에 append
tmp를 index_list에 담는다.

만약 공백이 존재하지 않으면? --> 단일 단어
바로 index_word에 사전참조하여 [리스트에 담아서] index_list에 담는다.

index_list 반환

"""

def contexts_to_index(sentence_list):
    index_list = []
    for sentence in sentence_list:
        if " " in sentence:
            tmp = []
            word_list = sentence.split()
            for word in word_list:
                try:
                    tmp.append(word_index[word])
                except:
                    try:
                        tmp.append(word_index[word.strip()])
                    except:
                        pass
                    # 만약 안되면 추가
                    tmp.append(0)
            index_list.append(tmp)
        else:
            index_list.append([word_index[sentence]])
    return index_list

"""
make_full_text(sentence_list):
input:
output:


"""

def make_full_text(sentence_list):
    full_text = ''
    for sentence in sentence_list:
        full_text += sentence + ' '
    return full_text.strip()


def vectorize_sequences(sequences, dimension=10000):
    # 크기가 (len(sequences), dimension))이고 모든 원소가 0인 행렬을 만듭니다
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # results[i]에서 특정 인덱스의 위치를 1로 만듭니다
    return results

In [7]:
text = make_sentences(X_train)

In [8]:
# 같은 과정으로 테스트케이스도 데이터 변환을 해준다.
test_text  = make_sentences(X_test)

In [9]:
data = pd.DataFrame(text, columns=['text'])
data['label'] = y_train
data

,text,label
0,the as you with out themselves powerful lets l...,1
1,the thought solid thought senator do making to...,0
2,the as there in at by br of sure many br of pr...,0
3,the of bernadette mon they halfway of identity...,1
4,the sure themes br only acting i i was favouri...,0
...,...,...
24995,the as it is ludicrous on not rape br program ...,1
24996,the slaughter susan effects is following like ...,0
24997,the this is anything tv tormented it is genera...,0
24998,the bar reverse me we endearing was song deep ...,1


In [10]:
# 훈련데이터로 전체 텍스트 만들기
full_text = make_full_text(text)

In [11]:

"""
훈련데이터에서
가장 많이 사용된 단어 10000개 찾기
"""
counter = Counter(full_text.split())
common_word =[i for i,j in counter.most_common(10000)]

# 유니크하게 바꾸기 (중복제거)
unique_word = list(set(common_word))

# enumerate로 새로운 word_index 사전 생성
index_word = { k+1:v for k, v in enumerate(unique_word)}
word_index = {v:k for k,v in index_word.items()}

In [12]:
indexed_sentece = contexts_to_index(data['text'])

In [13]:
# 같은 과정으로 테스트케이스도 데이터 변환을 해준다.
test_sentence = contexts_to_index(test_text)

In [20]:
train = vectorize_sequences(indexed_sentece,10001)
# 같은 과정으로 테스트케이스도 데이터 변환을 해준다.
test = vectorize_sequences(test_sentence,10001)

In [15]:
x_val = train[:1000]
partial_x_train = train[1000:]

y_val = y_train[:1000]
partial_y_train = y_train[1000:]

In [16]:
input_shape = x_val[0].shape

network = models.Sequential()
network.add(layers.Dense(64,activation='relu', input_shape = input_shape))
network.add(layers.Dense(256,activation ='relu'))
network.add(layers.Dropout(0.25))
network.add(layers.Dense(128,activation ='relu'))
network.add(layers.Dropout(0.25))
network.add(layers.Dense(64,activation ='elu'))
network.add(layers.Dropout(0.25))
network.add(layers.Dense(32,activation ='elu'))
network.add(layers.Dropout(0.25))
network.add(layers.Dense(16,activation ='relu'))
network.add(layers.Dense(1, activation='sigmoid'))

2022-05-03 14:04:07.376882: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
optimizer='adam'
loss='binary_crossentropy'
metrics=['accuracy']

network.compile(optimizer=optimizer,
              loss=loss,
              metrics=metrics)

In [18]:
epochs = 10
batch_size = 400

history = network.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(x_val, y_val))

2022-05-03 14:04:07.884093: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 960096000 exceeds 10% of free system memory.


Epoch 1/10
60/60 [==============================] - 2s 19ms/step - loss: 0.4131 - accuracy: 0.8102 - val_loss: 0.2753 - val_accuracy: 0.8880
Epoch 2/10
60/60 [==============================] - 1s 19ms/step - loss: 0.1991 - accuracy: 0.9287 - val_loss: 0.2742 - val_accuracy: 0.8980
Epoch 3/10
60/60 [==============================] - 1s 18ms/step - loss: 0.1231 - accuracy: 0.9603 - val_loss: 0.3195 - val_accuracy: 0.8770
Epoch 4/10
60/60 [==============================] - 1s 18ms/step - loss: 0.0641 - accuracy: 0.9807 - val_loss: 0.4115 - val_accuracy: 0.8800
Epoch 5/10
60/60 [==============================] - 1s 18ms/step - loss: 0.0350 - accuracy: 0.9887 - val_loss: 0.5604 - val_accuracy: 0.8760
Epoch 6/10
60/60 [==============================] - 1s 17ms/step - loss: 0.0276 - accuracy: 0.9910 - val_loss: 0.5406 - val_accuracy: 0.8900
Epoch 7/10
60/60 [==============================] - 1s 18ms/step - loss: 0.0124 - accuracy: 0.9964 - val_loss: 0.6903 - val_accuracy: 0.8790
Epoch 8/10
60

In [26]:
network.evaluate(test,y_test)

782/782 [==============================] - 1s 1ms/step - loss: 1.0231 - accuracy: 0.8578


[1.02308189868927, 0.8577600121498108]